In [8]:
import sys
from pathlib import Path

# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
sys.path.insert(0, str(PROJECT_ROOT))

# Python 내장 code 모듈 캐시만 임시 제거
if 'code' in sys.modules:
    del sys.modules['code']

# 이제 import
# from code.loading import DataLoader
from code.utils.statistic import Chi2Test

# loader2 = DataLoader(
#     start=2020,
#     end=2025,
#     output_file=DATA_DIR / 'maude_sample.parquet',
#     max_workers=4
# )

In [9]:
# adapter = 'polars'
# polars_kwargs = {
#     'use_statistics': True,
#     'parallel': 'auto',
#     'low_memory': False,
#     'rechunk': False,
#     'cache': True,
# }

# df = loader2.load(adapter=adapter, **polars_kwargs)

In [10]:
import polars as pl
import pandas as pd

In [11]:
# import sys
# from pathlib import Path

# sys.path.append(str(Path.cwd().parent / "code"))
# from utils.statistic import Chi2Test

In [12]:
import polars as pl

df = pl.read_csv(
    "maude_extracted_sample2.csv",
    infer_schema_length=0
) # 일단 모든 data 문자열로 받음

# 고장 유형 탐색

현재 mdr_text 전처리가 완료되어있지 않은 상황이라 가상의 파생변수를 두고 EDA 코드를 작성할 예정

아래는 결함 유형(고장 유형, manufacturer_inspection)의 종류이다:
* FUNCTIONAL_FAILURE = "Functional Failure"
* MECHANICAL_STRUCTURAL = "Mechanical/Structural"
* ELECTRICAL_POWER = "Electrical/Power"
* SOFTWARE_INTERFACE = "Software/Interface"
* ALARM_ALERT = "Alarm/Alert"
* SENSOR_ACCURACY = "Sensor/Accuracy"
* COMMUNICATION_CONNECTIVITY = "Communication/Connectivity"
* LABELING_PACKAGING = "Labeling/Packaging"
* STERILITY_CONTAMINATION = "Sterility/Contamination"
* USER_HUMAN_FACTOR = "User/Human Factor"
* ENVIRONMENTAL_COMPATIBILITY = "Environmental/Compatibility"
* OTHER = "Other"
* UNKNOWN = "Unknown"

product code 필드명: device_*_device_report_product_code
 

## 이상 사례 발생 top 10 제품코드 / 결함 유형 발생 비율

In [13]:
# 전체 제품 코드에서 이상 사례 발생 수가 높은 상위 10개 제품 코드 확인
top10_result = (
    df
    .group_by("device_0_device_report_product_code")
    .agg(pl.count().alias("total_count"))
    .sort("total_count", descending=True)
    .head(10)
    # .collect()
)
print(top10_result)

# 상위 10개 제품 코드 리스트화
top10_product_codes = top10_result['device_0_device_report_product_code'].to_list()

shape: (10, 2)
┌─────────────────────────────────┬─────────────┐
│ device_0_device_report_product… ┆ total_count │
│ ---                             ┆ ---         │
│ str                             ┆ u32         │
╞═════════════════════════════════╪═════════════╡
│ OZP                             ┆ 228         │
│ LGW                             ┆ 55          │
│ PQF                             ┆ 50          │
│ FTR                             ┆ 49          │
│ LWS                             ┆ 48          │
│ DTB                             ┆ 41          │
│ OZD                             ┆ 37          │
│ NVN                             ┆ 37          │
│ OYC                             ┆ 33          │
│ MKJ                             ┆ 26          │
└─────────────────────────────────┴─────────────┘


/var/folders/jz/7b91vg4d00z75sw7vt8q3c3h0000gn/T/ipykernel_5680/2594393994.py:5: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  .agg(pl.count().alias("total_count"))


In [14]:
# 전체 기기 이상 사례 데이터에서 각 결함 유형의 발생 비율 확인
ratio_result = (
    df
    .group_by("defect_type")  # 결함 유형별로 그룹화
    .agg(pl.len().alias("count"))
    .with_columns(
        (pl.col("count") / pl.col("count").sum() * 100)
        .round(2)
        .alias("percentage")
    )
    .sort("percentage", descending=True)
    # .collect()  # LazyFrame이면 필요
)
print(ratio_result.to_pandas())

                    defect_type  count  percentage
0                         Other    294        29.4
1              Electrical/Power    152        15.2
2         Mechanical/Structural    132        13.2
3               Sensor/Accuracy    119        11.9
4    Communication/Connectivity     72         7.2
5            Functional Failure     58         5.8
6             User/Human Factor     42         4.2
7                          None     42         4.2
8                   Alarm/Alert     29         2.9
9            Software/Interface     20         2.0
10                      Unknown     14         1.4
11  Environmental/Compatibility     13         1.3
12      Sterility/Contamination     11         1.1
13           Labeling/Packaging      2         0.2


## 어떤 제품코드가 어떤 고장 유형이 많이 발생하는가?

In [15]:
# product_code 별로 manufacturer_inspection의 리스트와 count, percentage를 구하는 코드
product_code_result = (
    df
    .filter(pl.col("device_0_device_report_product_code").is_in(top10_product_codes))
    .group_by(["device_0_device_report_product_code", "defect_type"])
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
    .group_by("device_0_device_report_product_code")
    .head(3)  # 각 그룹별 상위 3개
    .with_columns(
        (pl.col("count") / pl.col("count").sum().over("device_0_device_report_product_code") * 100)
        .round(2)
        .alias("percentage")
    )
    .sort(["device_0_device_report_product_code", "percentage"], descending=[False, True])
    # .collect()
)
print(product_code_result)

shape: (30, 4)
┌─────────────────────────────────┬────────────────────────────┬───────┬────────────┐
│ device_0_device_report_product… ┆ defect_type                ┆ count ┆ percentage │
│ ---                             ┆ ---                        ┆ ---   ┆ ---        │
│ str                             ┆ str                        ┆ u32   ┆ f64        │
╞═════════════════════════════════╪════════════════════════════╪═══════╪════════════╡
│ DTB                             ┆ Sensor/Accuracy            ┆ 18    ┆ 52.94      │
│ DTB                             ┆ Other                      ┆ 13    ┆ 38.24      │
│ DTB                             ┆ Electrical/Power           ┆ 3     ┆ 8.82       │
│ FTR                             ┆ Other                      ┆ 29    ┆ 69.05      │
│ FTR                             ┆ Mechanical/Structural      ┆ 9     ┆ 21.43      │
│ …                               ┆ …                          ┆ …     ┆ …          │
│ OZP                             ┆ Ele

## 어느 제조사에서 어떤 고장 유형이 많이 발생하는가?

In [22]:
# manufacturer별로 manufacturer_inspection의 리스트와 count, percentage를 구하는 코드
manufacturer_result = (
    df
    .filter(pl.col("manufacturer_g1_name").is_not_null())
    .group_by(["manufacturer_g1_name", "defect_type"])
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
    .group_by("manufacturer_g1_name")
    .head(5)  # 각 그룹별 상위 5개
    .with_columns(
        (pl.col("count") / pl.col("count").sum().over("manufacturer_g1_name") * 100)
        .round(2)
        .alias("percentage")
    )
    .sort(["manufacturer_g1_name", "percentage"], descending=[False, True])
    # .collect()
)
print(manufacturer_result)

shape: (190, 4)
┌────────────────────────────────┬───────────────────────┬───────┬────────────┐
│ manufacturer_g1_name           ┆ defect_type           ┆ count ┆ percentage │
│ ---                            ┆ ---                   ┆ ---   ┆ ---        │
│ str                            ┆ str                   ┆ u32   ┆ f64        │
╞════════════════════════════════╪═══════════════════════╪═══════╪════════════╡
│ ABBOTT                         ┆ Sensor/Accuracy       ┆ 4     ┆ 44.44      │
│ ABBOTT                         ┆ Other                 ┆ 2     ┆ 22.22      │
│ ABBOTT                         ┆ Software/Interface    ┆ 1     ┆ 11.11      │
│ ABBOTT                         ┆ Functional Failure    ┆ 1     ┆ 11.11      │
│ ABBOTT                         ┆ Electrical/Power      ┆ 1     ┆ 11.11      │
│ …                              ┆ …                     ┆ …     ┆ …          │
│ ZOLL MANUFACTURING CORPORATION ┆ Electrical/Power      ┆ 5     ┆ 27.78      │
│ ZOLL MANUFACTURING COR

In [17]:
manufacturer_result.filter((pl.col("count") >= 3) & (~pl.col("defect_type").is_in(["Unknown", "Other"]))).unique("manufacturer_g1_name")

manufacturer_g1_name,defect_type,count,percentage
str,str,u32,f64
"""ABBOTT""","""Sensor/Accuracy""",4,44.44
"""ABBOTT MEDICAL""","""Functional Failure""",3,27.27
"""ABBOTT VASCULAR, REG # 3005718…","""User/Human Factor""",9,52.94
"""ABIOMED, INC.""","""Mechanical/Structural""",7,23.33
"""ALLERGAN (COSTA RICA)""","""Electrical/Power""",4,11.76
…,…,…,…
"""MPRI""","""Sensor/Accuracy""",31,57.41
"""ST. JUDE MEDICAL - NEUROMODULA…","""Sensor/Accuracy""",4,36.36
"""ST. JUDE MEDICAL, INC.(CRM-SYL…","""Sensor/Accuracy""",8,26.67


# 통계 분석

## 1) 카이제곱검정
- **목적**: 특정 제품코드에서 특정 결함이 과대표되는지 검증
- **귀무가설**: 결함 유형과 제품코드는 독립적이다
- **적용**:
    - 관측 빈도 vs 기대 빈도
    - 제품코드 A에서 '배터리 결함' 발생 빈도가 전체 평균 대비 유의하게 높은가?
- **결과 해석**: p < 0.05이면 특정 패턴 존재

## 2) 검정 이후 표준화 잔차 확인

- **목적**: 어떤 셀이 기대치를 크게 벗어나는지 정량화
- **공식**: `(관측값 - 기대값) / sqrt(기대값)`
- **기준**: |z| > 2이면 주목, |z| > 3이면 매우 이례적

## 3) 효과 크기 - Cramer’s V

- **목적**: 통계적 유의성과 별개로 실질적 연관 강도 측정
- **범위**: 0(독립) ~ 1(완전 연관)

In [18]:
# 상위 10개 제품만 필터링 + Pandas로 변환
df_filtered = (
    df
    .filter(
        (pl.col("device_0_device_report_product_code").is_in(top10_product_codes))
        & (~pl.col("defect_type").is_in(["Unknown", "Other"])))
    .select(["device_0_device_report_product_code", "defect_type"])
    # .collect()
    .to_pandas()  # Pandas로 변환
)

In [19]:
chi2_tester = Chi2Test()
chi2_result = chi2_tester.execute(df_filtered, 'device_0_device_report_product_code', 'defect_type')


[기대빈도 확인]
----------------------------------------
최소 기대빈도: 0.08
5 미만 셀: 81/110 (73.6%)
⚠️ 주의: Fisher's exact test 사용 권장
   (기대빈도가 너무 작아 카이제곱 검정 부정확)

[기대빈도]
defect_type                          Alarm/Alert  Communication/Connectivity  \
device_0_device_report_product_code                                            
DTB                                         1.29                        3.67   
FTR                                         0.78                        2.23   
LGW                                         1.98                        5.64   
LWS                                         1.80                        5.11   
MKJ                                         1.01                        2.88   
NVN                                         1.34                        3.80   
OYC                                         1.15                        3.28   
OZD                                         1.11                        3.15   
OZP                                      

defect_type,Alarm/Alert,Communication/Connectivity,Electrical/Power,Environmental/Compatibility,Functional Failure,Labeling/Packaging,Mechanical/Structural,Sensor/Accuracy,Software/Interface,Sterility/Contamination,User/Human Factor
device_0_device_report_product_code,,,,,,,,,,,
DTB,-1.14,-1.39,-1.42,-0.82,-1.54,-0.37,-0.95,5.20,0.05,1.40,-0.14
FTR,0.24,-0.82,-0.02,-0.64,-1.20,-0.29,3.31,-1.31,-0.76,-0.41,0.36
LGW,-1.41,0.57,2.12,-1.02,-0.86,-0.46,-1.40,-0.23,0.45,-0.65,0.92
LWS,-1.34,-2.26,-1.08,-0.97,-0.72,-0.44,-1.19,5.76,-0.28,2.63,-1.27
MKJ,-0.01,-1.70,1.21,-0.73,5.22,2.73,-1.02,-2.11,0.29,-0.46,-0.95
NVN,-1.16,-1.44,-1.86,-0.84,1.62,-0.38,0.71,2.96,-0.99,-0.53,-0.18
OYC,1.72,-1.26,0.43,1.79,-0.08,-0.35,2.51,-2.24,-0.92,-0.49,-1.02
OZD,-1.05,-1.77,-1.97,-0.76,2.77,-0.34,1.70,-1.29,0.20,-0.48,5.03
OZP,2.55,-2.21,2.32,2.47,-0.86,0.38,0.39,-2.70,1.47,-1.17,-0.75


(|잔차| > 2: 유의한 차이, |잔차| > 3: 매우 강한 연관성)

[결론]
✅ p-value(0.0000) < 0.05 → 귀무가설 기각
   device_0_device_report_product_code과(와) defect_type은(는) 관련이 있음   효과 크기: 중간 관계

[사후분석]
   표준화 잔차 |값| > 2인 셀 해석:
   표준화 잔차 |값| > 2인 셀 해석:
   • DTB - Sensor/Accuracy: 예상보다 많음 (잔차=5.20)
   • FTR - Mechanical/Structural: 예상보다 많음 (잔차=3.31)
   • LGW - Electrical/Power: 예상보다 많음 (잔차=2.12)
   • LWS - Communication/Connectivity: 예상보다 적음 (잔차=-2.26)
   • LWS - Sensor/Accuracy: 예상보다 많음 (잔차=5.76)
   • LWS - Sterility/Contamination: 예상보다 많음 (잔차=2.63)
   • MKJ - Functional Failure: 예상보다 많음 (잔차=5.22)
   • MKJ - Labeling/Packaging: 예상보다 많음 (잔차=2.73)
   • MKJ - Sensor/Accuracy: 예상보다 적음 (잔차=-2.11)
   • NVN - Sensor/Accuracy: 예상보다 많음 (잔차=2.96)
   • OYC - Mechanical/Structural: 예상보다 많음 (잔차=2.51)
   • OYC - Sensor/Accuracy: 예상보다 적음 (잔차=-2.24)
   • OZD - Functional Failure: 예상보다 많음 (잔차=2.77)
   • OZD - User/Human Factor: 예상보다 많음 (잔차=5.03)
   • OZP - Alarm/Alert: 예상보다 많음 (잔차=2.55)
   • OZP - Communication/Connectivity: 예상보다 적음 (

## 오즈비
- **목적**: 특정 제품에서 특정 결함 발생 확률 비교
- **공식**: `OR = (제품A 결함발생 오즈) / (다른 제품 결함발생 오즈)`
- **결과 예시**: "인슐린 펌프 X의 배터리 결함 OR = 4.2 (95% CI: 3.1-5.7)"
    - 해석: 다른 제품 대비 4.2배 높은 확률
    - 이후 다시 **카이제곱 검정**으로 특정 제품 대비 전체 제품 오즈비 비교가 유의한지 확인도 가능

In [20]:
import numpy as np
import pandas as pd

def calculate_odds_ratio(
    contingency_table: pd.DataFrame,
    target_product: str,
    target_defect: str,
    chi2_p_value: float = None,
    alpha: float = 0.05
) -> dict:
    """
    분할표에서 특정 제품-결함 조합의 오즈비 계산
    
    Parameters
    ----------
    contingency_table : pd.DataFrame
        pd.crosstab으로 생성한 분할표
        index: 제품 코드, columns: 결함 유형
    target_product : str
        분석 대상 제품 코드
    target_defect : str
        분석 대상 결함 유형
    chi2_p_value : float, optional
        Chi2Test()의 p-value (제공 시 사용)
    alpha : float
        유의수준 (default: 0.05)
    
    Returns
    -------
    dict
        - odds_ratio: 오즈비
        - ci_lower: 95% 신뢰구간 하한
        - ci_upper: 95% 신뢰구간 상한
        - p_value: 카이제곱 검정 p-value
        - interpretation: 해석
        - a, b, c, d: 2x2 분할표 셀 값
    
    Example
    -------
    >>> contingency_table = pd.crosstab(df['product_code'], df['defect_type'])
    >>> chi2_result = chi2_tester.execute(df, 'product_code', 'defect_type')
    >>> result = calculate_odds_ratio(contingency_table, 'INS', '배터리결함', chi2_result.p_value)
    >>> print(f"OR = {result['odds_ratio']:.2f} (95% CI: {result['ci_lower']:.2f}-{result['ci_upper']:.2f})")
    OR = 4.17 (95% CI: 2.52-6.89)
    
    Notes
    -----
    자동으로 2x2 분할표 생성:
                    타겟 결함    기타 결함
    타겟 제품          a           b
    기타 제품          c           d
    """
    
    # a: 타겟 제품 & 타겟 결함
    a = contingency_table.loc[target_product, target_defect] if target_product in contingency_table.index and target_defect in contingency_table.columns else 0
    
    # b: 타겟 제품 & 기타 결함
    b = contingency_table.loc[target_product, :].sum() - a if target_product in contingency_table.index else 0
    
    # c: 기타 제품 & 타겟 결함
    c = contingency_table.loc[:, target_defect].sum() - a if target_defect in contingency_table.columns else 0
    
    # d: 기타 제품 & 기타 결함
    total = contingency_table.values.sum()
    d = total - a - b - c
    
    print(f"\n[2x2 분할표: {target_product} - {target_defect}]")
    print(pd.DataFrame(
        [[a, b], [c, d]],
        index=[f'{target_product}', '기타 제품'],
        columns=[f'{target_defect}', '기타 결함']
    ))
    print()
    
    # 0 처리 (연속성 수정)
    if b == 0 or c == 0 or a == 0 or d == 0:
        a, b, c, d = a + 0.5, b + 0.5, c + 0.5, d + 0.5
    
    # 오즈비 계산
    odds_ratio = (a * d) / (b * c)
    
    # 로그 오즈비의 표준오차 (SE)
    se_log_or = np.sqrt(1/a + 1/b + 1/c + 1/d)
    
    # 95% 신뢰구간 (로그 스케일에서 계산 후 역변환)
    z_critical = 1.96  # 95% CI
    log_or = np.log(odds_ratio)
    ci_lower = np.exp(log_or - z_critical * se_log_or)
    ci_upper = np.exp(log_or + z_critical * se_log_or)
    
    # p-value (Chi2Test 결과 사용)
    p_value = chi2_p_value
    
    # 해석
    if p_value is not None and p_value < alpha:
        if odds_ratio > 1:
            interpretation = f"'{target_product}'에서 '{target_defect}'이(가) 다른 제품 대비 {odds_ratio:.2f}배 높음 (p={p_value:.4f}, 유의함)"
        else:
            interpretation = f"'{target_product}'에서 '{target_defect}'이(가) 다른 제품 대비 {1/odds_ratio:.2f}배 낮음 (p={p_value:.4f}, 유의함)"
    elif p_value is not None:
        interpretation = f"오즈비가 유의하지 않음 (p={p_value:.4f})"
    else:
        interpretation = "p-value 미제공"
    
    return {
        'odds_ratio': odds_ratio,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'p_value': p_value,
        'interpretation': interpretation,
        'a': int(a) if a == int(a) else a,
        'b': int(b) if b == int(b) else b,
        'c': int(c) if c == int(c) else c,
        'd': int(d) if d == int(d) else d
    }

# # 사용 예시
# if __name__ == "__main__":
#     # 예시 분할표 생성
#     data = {
#         '배터리결함': [50, 30, 20],
#         '화상': [100, 80, 60],
#         '오작동': [50, 40, 30]
#     }
#     contingency_table = pd.DataFrame(data, index=['INS', 'PUM', 'DEF'])
    
#     print("전체 분할표:")
#     print(contingency_table)
    
#     # 오즈비 계산
#     result = calculate_odds_ratio(
#         contingency_table=contingency_table,
#         target_product='INS',
#         target_defect='배터리결함',
#         chi2_p_value=0.0001
#     )
    
#     print(f"OR = {result['odds_ratio']:.2f}")
#     print(f"95% CI: ({result['ci_lower']:.2f}, {result['ci_upper']:.2f})")
#     print(f"해석: {result['interpretation']}")

In [24]:
# 1. 분할표 생성
contingency_table = pd.crosstab(
    df_filtered['device_0_device_report_product_code'], 
    df_filtered['defect_type']
)

# 2. 카이제곱 검정 생략: chi2_result 이미 계산됨

# 3. 오즈비 계산 (분할표 + 타겟만 지정)
result = calculate_odds_ratio(
    contingency_table=contingency_table,
    target_product='OZP',      # 분석할 제품 코드
    target_defect='Sensor/Accuracy',  # 분석할 결함 유형
    chi2_p_value=chi2_result.p_value
)

print(f"OR = {result['odds_ratio']:.2f} (95% CI: {result['ci_lower']:.2f}-{result['ci_upper']:.2f})")


[2x2 분할표: OZP - Sensor/Accuracy]
       Sensor/Accuracy  기타 결함
OZP                 14    127
기타 제품               69    202

OR = 0.32 (95% CI: 0.17-0.60)
